## 4 命題論理の基礎



### 4.1 命題論理について



### 4.2 命題論理式の構文



命題変数の定義の例



In [1]:
from cspsat import *
p = Bool("p")
q = Bool("q")
print([p, q])

In [1]:
[p(1), q(2,3), q("a"), q([1,2])]

In [1]:
[ p(i) for i in range(5) ]

命題論理式の記述の例



In [1]:
(p, q, r) = (Bool("p"), Bool("q"), Bool("r"))
["or", ["and", p, q], r]

#### 練習問題 4.1.2: 例題「嘘つきパズル」の発言をcspsat記法で表す



In [1]:
(A, B) = (Bool("A"), Bool("B"))
print(["not", B])
print(["and", A, B])

### 4.3 命題論理式の意味論



真理値表の表示の例



In [1]:
from cspsat import *
(p, q) = (Bool("p"), Bool("q"))
truthTable(["imp",p,q], ["equ",p,q], ["xor",p,q])

In [1]:
truthTable(["imp",p,q], ["or",["not",p],q])

In [1]:
truthTable(["imp",p,["imp",q,p]])

In [1]:
truthTable(["and",["equ",p,q],["xor",p,q]])

命題論理式のモデルの表示の例



In [1]:
for model in models(["xor",p,q], num=0):
    print(model)

#### 練習問題 4.2.1: 恒真か充足可能か充足不能か



In [1]:
(p, q) = (Bool("p"), Bool("q"))
f1 = ["imp", p, p]
f2 = ["imp", p, ["not", p]]
f3 = ["equ", p, ["not", p]]
f4 = ["imp", ["imp", ["imp", p, q], p], p]
truthTable(f1, f2, f3)
print()
truthTable(f4)

#### 練習問題 4.2.2: 対偶と逆



In [1]:
(p, q) = (Bool("p"), Bool("q"))
f1 = ["imp", p, q]
f2 = ["imp", ["not", q], ["not", p]]
f3 = ["imp", q, p]
truthTable(f1, f2, f3)

### 4.4 命題論理式の同値変形



#### 練習問題 4.3.2: 多数決回路



In [1]:
(p, q, r) = (Bool("p"), Bool("q"), Bool("r"))
f1 = ["or",["and",p,q],["and",p,r],["and",q,r]]
f2 = ["and",["or",p,q],["or",p,r],["or",q,r]]
truthTable(f1, f2)

### 4.5 CNF式



#### 4.5.1 補足



同値なCNF式の例



In [1]:
from cspsat import *
(p, q) = (Bool("p"), Bool("q"))
f1 = ["and", ["or", p, ~q], ["or", ~p, q], ["or", ~p, ~q]]
f2 = ["and", ["not",p], ["not",q]]
truthTable(f1, f2)

### 4.6 CNF式への変換



toNF関数による標準形への変換の例



In [1]:
from cspsat import *
(p, q) = (Bool("p"), Bool("q"))
f = ["equ", ["imp", p, q], ["xor", p, q]]
toNF(f)

toNNF関数による否定標準形への変換の例



In [1]:
toNNF(f)

toCNF関数によるCNF式(論理積標準形)への変換の例 (simplify処理なし)



In [1]:
toCNF(f, simplify=False)

toCNF関数によるCNF式(論理積標準形)への変換の例 (simplify処理あり)



In [1]:
toCNF(f)

solveSAT関数によるCNF式のモデル探索の例



In [1]:
solveSAT(toCNF(f))

#### 4.6.1 補足



In [1]:
from cspsat import *
p = Bool("p")
cnf = [ [p(1),p(2)], [~p(1),~p(2)] ]
saveSAT(cnf, "foo.sat")
cnf = loadSAT("foo.sat")
solveSAT(cnf)

In [1]:
p = Bool("p")
cnf = [ [p(1),p(2)], [~p(1),~p(2)] ]
saveDimacs(cnf, "foo.cnf")
! ./sat4j foo.cnf

##### 練習問題 4.5.2: Pi関数



Pi関数のCNF式



In [1]:
(w, x, y, z) = (Bool("w"), Bool("x"), Bool("y"), Bool("z"))
pi = ["or", ["and", ~w, ~x, ~y], ["and", ~w,  x, ~y, ~z],
                        ["and", ~w,  x,  y,  z], ["and",  w,  x] ]
print(toCNF(pi))

Pi関数の真理値表



In [1]:
truthTable(pi)

### 4.7 導出原理



例題「コイン投げ」のCNF式



In [1]:
from cspsat import *
(h, t, m, y) = (Bool("h"), Bool("t"), Bool("m"), Bool("y"))
f = ["and", ["imp",h,m], ["imp",t,~y], ["xor",h,t], ["xor",m,y]]
toCNF(f)

例題「コイン投げ」の解



In [1]:
solveSAT(toCNF(f), num=0)

例題「架空の国の国民の好物」のCNF式 (リベストのCNF式)



In [1]:
def rivestCNF():
    (a, b, c, d) = (Bool("a"), Bool("b"), Bool("c"), Bool("d"))
    f1 = ["imp", ["and",~a,~b], ~c]
    f2 = ["imp", d, ["or",b,c]]
    f3 = ["not", ["and",~a,~c,~d]]
    f4 = ["imp", ["and",~d,a], b]
    f5 = ["imp", ["and",a,b], c]
    f6 = ["imp", ~d, ["or",~b,~c]]
    f7 = ["not", ["and",a,c,d]]
    f8 = ["imp", ["and",b,d], a]
    return toCNF(["and", f1, f2, f3, f4, f5, f6, f7, f8])
rivestCNF()

例題「架空の国の国民の好物」の解 (リベストのCNF式)



In [1]:
solveSAT(rivestCNF())

### 4.8 SATソルバーの原理



DPLLアルゴリズムの動作の例



In [1]:
from cspsat import *
dpll = DPLL(verbose=1)
dpll.solve(rivestCNF())